Database ve tabloların oluşturulup kayıtların eklenmesi

In [3]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

# Bağlantı bilgileri
credentials = {
    "dbname": "company",
    "user": "postgres",
    "password": "12345",
    "host": "localhost",
    "port": "5432"
}

def create_database_if_not_exists(creds):
    # Önce postgres veritabanına bağlan
    conn = psycopg2.connect(
        dbname="postgres",
        user=creds["user"],
        password=creds["password"],
        host=creds["host"],
        port=creds["port"]
    )
    conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    cur = conn.cursor()
    try:
        cur.execute(f"CREATE DATABASE {creds['dbname']};")
        print(f"{creds['dbname']} veritabanı oluşturuldu.")
    except psycopg2.errors.DuplicateDatabase:
        print(f"{creds['dbname']} veritabanı zaten mevcut.")
    finally:
        cur.close()
        conn.close()

def create_tables_and_insert_data(creds):
    conn = psycopg2.connect(**creds)
    cur = conn.cursor()

    # Tablo oluşturma
    cur.execute("""
        CREATE TABLE IF NOT EXISTS departments (
            emp_id SERIAL PRIMARY KEY,
            dept_name VARCHAR(50) NOT NULL,
            dept_id INTEGER NOT NULL
        );
    """)
    cur.execute("""
        CREATE TABLE IF NOT EXISTS employees (
            emp_id INTEGER PRIMARY KEY,
            first_name VARCHAR(50),
            last_name VARCHAR(50),
            salary INTEGER,
            job_title VARCHAR(100),
            gender VARCHAR(10),
            hire_date DATE NOT NULL
        );
    """)

    # departments verileri
    departments_data = [
        (17679, "Operations", 13),
        (26650, "Marketing", 14),
        (30840, "Operations", 13),
        (49823, "Technology", 12),
        (51821, "Operations", 13),
        (67323, "Marketing", 14),
        (71119, "Administrative", 11),
        (76589, "Operations", 13),
        (97927, "Technology", 12)
    ]

    # employees verileri
    employees_data = [
        (17679, "Robert", "Gilmore", 110000, "Operations Director", "Male", "2018-09-04"),
        (26650, "Elvis", "Ritter", 86000, "Sales Manager", "Male", "2017-11-24"),
        (30840, "David", "Barrow", 85000, "Data Scientist", "Male", "2019-12-02"),
        (49714, "Hugo", "Forester", 55000, "IT Support Specialist", "Male", "2019-11-22"),
        (51821, "Linda", "Foster", 95000, "Data Scientist", "Female", "2019-04-29"),
        (67323, "Lisa", "Wiener", 75000, "Business Analyst", "Female", "2018-08-09"),
        (70950, "Rodney", "Weaver", 87000, "Project Manager", "Male", "2018-12-20"),
        (71329, "Gayle", "Meyer", 77000, "HR Manager", "Female", "2019-06-28"),
        (76589, "Jason", "Christian", 99000, "Project Manager", "Male", "2019-01-21"),
        (97927, "Billie", "Lanning", 67000, "Web Developer", "Female", "2018-06-25")
    ]

    # Kayıt ekleme (varsa hata vermemesi için ON CONFLICT DO NOTHING)
    cur.executemany("""
        INSERT INTO departments (emp_id, dept_name, dept_id)
        VALUES (%s, %s, %s)
        ON CONFLICT (emp_id) DO NOTHING;
    """, departments_data)

    cur.executemany("""
        INSERT INTO employees (emp_id, first_name, last_name, salary, job_title, gender, hire_date)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (emp_id) DO NOTHING;
    """, employees_data)

    conn.commit()
    cur.close()
    conn.close()

create_database_if_not_exists(credentials)
create_tables_and_insert_data(credentials)

company veritabanı zaten mevcut.


SQL Fetchall

In [59]:
def get_data_list(creds, query, params=None):
    """Verilen sorguyu çalıştırır ve sonuçları döndürür."""
    import psycopg2
    results = []
    try:
        with psycopg2.connect(**creds) as conn:
            with conn.cursor() as cur:
                cur.execute(query, params or ())
                results = cur.fetchall()
    except Exception as e:
        print("Veritabanı hatası:", e)
    return results

SQL Fetchone

In [60]:
def get_data(creds, query, params=None):
    """Verilen sorguyu çalıştırır ve sonuçları döndürür."""
    import psycopg2
    results = []
    try:
        with psycopg2.connect(**creds) as conn:
            with conn.cursor() as cur:
                cur.execute(query, params or ())
                results = cur.fetchone()
    except Exception as e:
        print("Veritabanı hatası:", e)
    return results

1 - Find the employees who get paid more than Rodney Weaver.

In [ ]:
sql_query = """
    SELECT e.first_name, e.last_name, e.salary
    FROM employees e
    CROSS JOIN (
        SELECT salary FROM employees
        WHERE first_name =  %s AND last_name =  %s
        LIMIT 1
    ) r
    WHERE e.salary > r.salary
    ORDER BY e.salary DESC;
"""

results = get_data_list(credentials, sql_query, ('Rodney', 'Weaver'))

for row in results:
    print(row)


('Robert', 'Gilmore', 110000)
('Jason', 'Christian', 99000)
('Linda', 'Foster', 95000)


2 - Find the average, min and max salaries

In [ ]:
sql_query = """
    SELECT 
        CAST(ROUND(AVG(salary),0) AS INTEGER) AS average_salary,
        MAX(salary) AS maximum_salary,
        MIN(salary) AS minimum_salary
    FROM employees;
"""

results = get_data(credentials, sql_query)

if results:
    print(f"Ortalama Maaş: {results[0]} \nMaksimum Maaş: {results[1]} \nMinimum Maaş: {results[2]}")

Ortalama Maaş: 83600 
Maksimum Maaş: 110000 
Minimum Maaş: 55000


3 - Find the employees whose salary is more than 87000. Our query should return first name, last name, and salary info of the employees.

In [ ]:
sql_query = """
    SELECT e.first_name, e.last_name, e.salary
    FROM employees e
    WHERE e.salary > 87000
    ORDER BY e.salary DESC;
"""

results = get_data_list(credentials, sql_query, ('Rodney', 'Weaver'))

for row in results:
    print(row[0] +" "+ row[1], row[2])

Robert Gilmore 110000
Jason Christian 99000
Linda Foster 95000


4 - Find the employees (first name, last name from employees table) who work under the Operations department (departments table). Our query should return first name and last name info.

In [ ]:
sql_query = """
    SELECT e.first_name, e.last_name
    FROM employees e
    JOIN departments d ON e.emp_id = d.emp_id
    WHERE d.dept_name = %s;
"""

results = get_data_list(credentials, sql_query, ("Operations",))

for row in results:
    print(row[0] +" "+ row[1])

Robert Gilmore
David Barrow
Linda Foster
Jason Christian


5 - Find the employees (first name, last name from employees table) who work under the Technology department (departments table). Our query should return first name and last name info.

In [ ]:
sql_query = """
    SELECT e.first_name, e.last_name
    FROM employees e
    JOIN departments d ON e.emp_id = d.emp_id
    WHERE d.dept_name = %s;
"""

results = get_data_list(credentials, sql_query, ("Technology",))

for row in results:
    print(row[0] +" "+ row[1])

Billie Lanning


6 - Find the average salary of female employees.

In [75]:
sql_query = """
    SELECT 
        CAST(ROUND(AVG(salary),0) AS INTEGER) AS average_salary
    FROM employees
    WHERE gender = %s; 
"""

results = get_data(credentials, sql_query, ('Female',))

if results:
    print(f"Averege salery of women: {results[0]} ")

Averege salery of women: 78500 


7 - Find the average salaries of each department.

In [79]:
sql_query = """
    SELECT 
        d.dept_name, CAST(ROUND(AVG(e.salary),0) AS INTEGER) AS average_salary
    FROM employees e
    JOIN departments d ON e.emp_id = d.emp_id
    GROUP BY d.dept_name;
"""

results = get_data_list(credentials, sql_query)

for row in results:
    print(f"{row[0]}  \t {row[1]}")

Technology  	 67000
Marketing  	 80500
Operations  	 97250


8 - Find the oldest and newest employees.

In [87]:
sql_query = """
    SELECT first_name, last_name, hire_date
    FROM employees
    WHERE hire_date = (SELECT MIN(hire_date) FROM employees)
       OR hire_date = (SELECT MAX(hire_date) FROM employees);
"""

results = get_data_list(credentials, sql_query)

for row in results:
    print(row[0] +" "+ row[1], row[2])

Elvis Ritter 2017-11-24
David Barrow 2019-12-02


9 - Find the hiring date and department of the highest paid employee

In [102]:
sql_query = """
    SELECT e.first_name, e.last_name, e.hire_date, d.dept_name
    FROM employees e
    JOIN departments d ON e.emp_id = d.emp_id
    WHERE e.salary = (SELECT MAX(salary) FROM employees);
"""

results = get_data(credentials, sql_query)

print(f"Name: {results[0]} {results[1]}\nHire Date: {results[2]} \nDepartment: {results[3]}")

Name: Robert Gilmore
Hire Date: 2018-09-04 
Department: Operations


10 - Find the hiring date and department of the lowest paid employee

In [116]:
sql_query = """
    SELECT e.first_name, e.last_name, e.hire_date, d.dept_name
    FROM employees e
    JOIN departments d ON e.emp_id = d.emp_id
    WHERE e.salary = (
        SELECT MIN(e2.salary)
        FROM employees e2
        JOIN departments d2 ON e2.emp_id = d2.emp_id
    );
"""

results = get_data_list(credentials, sql_query)

for row in results:
    print(f"Name: {row[0]} {row[1]}\nHire Date: {row[2]} \nDepartment: {row[3]}")

Name: Billie Lanning
Hire Date: 2018-06-25 
Department: Technology


In [ ]:
import csv
import os
import psycopg2

def read_csv_columns(file_path, columns):
    """
    Belirtilen CSV dosyasından, istenen sütunlardaki verileri tablo olarak döndürür.
    :param file_path: CSV dosyasının yolu
    :param columns: Almak istediğiniz sütun adlarının listesi
    :return: Seçilen sütunlardan oluşan satırların listesi
    """
    results = []
    with open(file_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            selected = [row[col] for col in columns]
            results.append(selected)
    return results


def insert_kursiyerler(creds, data):
    """
    data: [['Ad Soyad', 'Mail', 'Telefon', 'Posta Kodu', 'Eyalet'], ...]
    """
    conn = psycopg2.connect(**creds)
    cur = conn.cursor()
    insert_query = """
        INSERT INTO kursiyerler (AdSoyad, MailAdresi, TelefonNumarasi, PostaKodu, YasadiginizEyalet)
        VALUES (%s, %s, %s, %s, %s)
        ON CONFLICT DO NOTHING;
    """
    for row in data:
        cur.execute(insert_query, row)
    conn.commit()
    cur.close()
    conn.close()

# Kullanım örneği:
# Önce CSV'den verileri oku
file_path = 'Basvurular - Sayfa1.csv'
columns = ["Adınız Soyadınız", "Mail adresiniz", "Telefon Numaranız", "Posta Kodunuz", "Yaşadığınız Eyalet"]
liste = read_csv_columns(file_path, columns)

# Veritabanı bağlantı bilgileri
creds = {
    "dbname": "wearehere",
    "user": "postgres",
    "password": "12345",
    "host": "localhost",
    "port": "5432"
}

insert_kursiyerler(creds, liste)

In [ ]:
import csv
import psycopg2
from datetime import datetime

def parse_zaman_damgasi(zaman_str):
    """
    Farklı formatlardaki zaman damgalarını YYYY-MM-DD HH:MM:SS formatına çevirir.
    """
    zaman_str = zaman_str.strip()
    formats = [
        "%m/%d/%Y %H:%M:%S",   # 4/30/2023 21:38:19
        "%d.%m.%y %H:%M",      # 05.01.23 01:39
        "%d.%m.%y %H:%M:%S",   # 05.01.23 01:39:00 (varsa)
    ]
    for fmt in formats:
        try:
            return datetime.strptime(zaman_str, fmt).strftime("%Y-%m-%d %H:%M:%S")
        except ValueError:
            continue
    raise ValueError(f"ZamanDamgasi formatı tanınmadı: {zaman_str}")

def read_csv_for_basvurular(file_path):
    """
    CSV'den Basvurular tablosuna uygun verileri okur ve sıralı KursiyerID ekler.
    ZamanDamgasi alanını standart formata çevirir.
    """
    results = []
    with open(file_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile, delimiter='\t', skipinitialspace=True)
        kursiyer_id = 1
        for row in reader:
            data = [
                kursiyer_id,
                parse_zaman_damgasi(row["zamandamgasi"]),  # Standart formata çevir
                row["suankidurum"],
                row["itphegitimkatilmak"],
                row["ekonomikdurum"],
                row["dilkursunadevam"],
                row["ingilizceseviye"],
                row["hollandacaseviye"],
                row["baskigoruyor"],
                row["bootcampbitirdi"],
                row["onlineitkursu"],
                row["ittecrube"],
                row["projedahil"],
                row["calismakistegi"],
                row["nedenkatilmakistiyor"],
                row["basvurudonemi"],
            ]
            results.append(data)
            kursiyer_id += 1
    return results

def insert_basvurular(creds, data):
    """
    Basvurular tablosuna verileri ekler.
    """
    conn = psycopg2.connect(**creds)
    cur = conn.cursor()
    insert_query = """
        INSERT INTO Basvurular (
            KursiyerID, ZamanDamgasi, SuAnkiDurum, ITPHEgitimKatilmak, EkonomikDurum,
            DilKursunaDevam, IngilizceSeviye, HollandacaSeviye, BaskiGoruyor,
            BootcampBitirdi, OnlineITKursu, ITTecrube, ProjeDahil, CalismakIstegi,
            NedenKatilmakIstiyor, BasvuruDonemi
        )
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT DO NOTHING;
    """
    for row in data:
        cur.execute(insert_query, row)
    conn.commit()
    cur.close()
    conn.close()

# Kullanım örneği:
file_path = "Basvurular - Sayfa1.csv"
liste = read_csv_for_basvurular(file_path)

creds = {
    "dbname": "wearehere",
    "user": "postgres",
    "password": "12345",
    "host": "localhost",
    "port": "5432"
}

insert_basvurular(creds, liste)

KeyError: 'ZamanDamgasi'

In [13]:
import csv

with open("Basvurular - Sayfa1.csv", newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t')
    print(reader.fieldnames)

['ZamanDamgasi,Adınız Soyadınız,Mail adresiniz,Telefon Numaranız,Posta Kodunuz,Yaşadığınız Eyalet,suankidurum,itphegitimkatilmak,ekonomikdurum,dilkursunadevam,ingilizceseviye,hollandacaseviye,baskigoruyor,bootcampbitirdi,onlineitkursu,ittecrube,projedahil,calismakistegi,nedenkatilmakistiyor,basvurudonemi,mentorgorusmesi']
